---
title: "Data Collection"
format:
    html: 
        code-fold: false
---

{{< include instructions.qmd >}} 


{{< include overview.qmd >}} 

{{< include methods.qmd >}} 

In [ ]:
Below is documentation on the process to even get this to run. 
Im writing at 3 AM and been working this nonstop since 4pm. I put in more time for one tiny part of this project than a full workday.

First, Riot API was extremely fickle with internet network. It recgonized the georgetown WIFI networks and rejected it. At home, my personal wifi worked fine.                                         

First to get the API key:
Publish your project publicly on Github (github classroom causes this to fail). It can be blank. On the project, create a txt file called riot_verify to verify.
Then make a Riot games API account. 
Then submit an application to get approval for API key.
it will give you a random code you need to copy and paste into that riot verify.txt file you made ealier.
It should approve once it detects the URL but URL needs to be exact otherwise you will need to resubmit (It took me 5 tries). 
                                          
However, there is an additonal part of the process you need to go through:
Make a riot games account (seperate from API account).
Despite TFT being a mobile game, there was a bug I encountered that wont catch ID correctly and I was unable to synch my riot games API with my TFT mobile account.
I believe this was due to me making my API account first, and my game account seacond.    
Even with gamecenter turned off, shutting down and restarting my iphone, and playing multiple games to completion, detect requests reading, it failed.
Instead, you need to go to riot, download Riot Client.
Then Login. 
Then click on League of Lengeds (totally different game).
Then click on TFT.                                
Play a full game (cant quite or forfeit).
Once you get to results, the match must fully resolve and you need to sit on results for a bit to make it register.
But since my first ID wasnt working, i had to make a seperate account just for this.
Once i made a 2nd account, followed everything, my api scrapper script worked...somehow.
    
If you set up a mobile games account before hand, it likely wont work and you need to make another new account to do so.
This entire process took me about 13 hours. I litteraly made my 2nd account fuckthisapi out of sheer spite. 

PUUID= z7tXLLhq0AA5E2G4Jdzq88KOlSkOndkZ56mWJnwqAzT5IV_6EOYhU1_ZySsxZtyudA53apw2vs_p5Q
MatchID = NA1_5419735432
Riot ID = vornelix#819

## Data Collection from Riot Games API

Purpose: In the following code, we be collecting data from the Riot Games API using Python. 

Constraints of the script:
I am limiting the reigion is to NA
I am only looking Champion/grandmaster level players (these are the two highest level players can rank. I plan on looking at high level play).
Simple random samp of 250 Champion and Grandmaster players (I wanted the diff ranks to be equal size but simp rand samps for analyis).
I am only looking at 25 matches played by the 500 players.
API has run limitations

Output is located in data/raw-data

This script will take quite a while to run and the API has a limit.
```python

In [46]:
#this is a test to use your API keey and the match you played
#if this script fails then the API scrapping script wont work
import requests

API_KEY = "RGAPI-1f9dca75-502e-4aec-8c6b-a2c6f2793778"
match_id = "NA1_5419735432"

url = f"https://americas.api.riotgames.com/tft/match/v1/matches/{match_id}"
res = requests.get(url, params={"api_key": API_KEY})

print("STATUS:", res.status_code)
print(res.text[:3000])

STATUS: 200
{"metadata":{"data_version":"6","match_id":"NA1_5419735432","participants":["z7tXLLhq0AA5E2G4Jdzq88KOlSkOndkZ56mWJnwqAzT5IV_6EOYhU1_ZySsxZtyudA53apw2vs_p5Q"]},"info":{"endOfGameResult":"GameComplete","gameCreation":1763881823000,"gameId":5419735432,"game_datetime":1763883928853,"game_length":2086.93115234375,"game_version":"Linux Version 15.23.728.3286 (Nov 21 2025/16:26:55) [PUBLIC] <Releases/15.23>","mapId":22,"participants":[{"companion":{"content_ID":"5897ad9f-4665-4372-8f3e-6c878adb8918","item_ID":1,"skin_ID":1,"species":"PetTFTAvatar"},"gold_left":38,"last_round":37,"level":9,"missions":{"PlayerScore2":207},"placement":1,"players_eliminated":3,"puuid":"z7tXLLhq0AA5E2G4Jdzq88KOlSkOndkZ56mWJnwqAzT5IV_6EOYhU1_ZySsxZtyudA53apw2vs_p5Q","riotIdGameName":"vornelix","riotIdTagline":"819","time_eliminated":2078.66748046875,"total_damage_to_players":267,"traits":[{"name":"TFT15_Bastion","num_units":2,"style":1,"tier_current":1,"tier_total":3},{"name":"TFT15_Captain","num_units"

In [47]:
#alt test script I tried
#if #is 200, and there is a summonerLevel, you are good. Otherwise, the API scrapping script wont work.
import requests

API_KEY = "RGAPI-1f9dca75-502e-4aec-8c6b-a2c6f2793778"
puuid = "6xBNi2AWs8C64Hy68JWN0WYe22Oa1VtAX_hdITn7WDS2Le-1UrCs2baBjj7VcAbfKMSrk5rIJuuPHg"

url = f"https://na1.api.riotgames.com/tft/summoner/v1/summoners/by-puuid/{puuid}"
res = requests.get(url, params={"api_key": API_KEY})

print(res.status_code)
print(res.text)

200
{"puuid":"6xBNi2AWs8C64Hy68JWN0WYe22Oa1VtAX_hdITn7WDS2Le-1UrCs2baBjj7VcAbfKMSrk5rIJuuPHg","profileIconId":29,"revisionDate":1763858444000,"summonerLevel":1}


In [ ]:
#Below is the entire raw data extraction script.

#import packages
import os
import time
import json
import random
import logging
import requests
from pathlib import Path

#CONFIG vars
API_KEY = "RGAPI-1f9dca75-502e-4aec-8c6b-a2c6f2793778" #when publish to Github, it should be PLACEHOLDER
REGION = "americas"   # For match endpoints
PLATFORM = "na1"      # For league & summoner endpoints

MATCHES_PER_PLAYER = 25
SAVE_DIR = Path("data/raw_matches")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

logging.basicConfig(filename="tft_scraper.log",
                    level=logging.INFO,
                    format="%(asctime)s [%(levelname)s] %(message)s")

#BASIC REQUEST WRAPPER WITH 429 RETRY
def riot_get(url, params=None):
    if params is None:
        params = {}
    params["api_key"] = API_KEY

    while True:
        r = requests.get(url, params=params)

        if r.status_code == 429:
            retry = int(r.headers.get("Retry-After", 1))
            logging.warning(f"429 hit. Sleeping {retry}s.")
            time.sleep(retry)
            continue

        return r

#GET Challenger PUUID list
def get_challenger_puuids():
    url = f"https://{PLATFORM}.api.riotgames.com/tft/league/v1/challenger"
    r = riot_get(url)
    data = r.json()

    entries = data.get("entries", [])
    return [e["puuid"] for e in entries if e.get("puuid")]

# GET Grandmaster PUUID list
def get_grandmaster_puuids():
    url = f"https://{PLATFORM}.api.riotgames.com/tft/league/v1/grandmaster"
    r = riot_get(url)
    data = r.json()

    entries = data.get("entries", [])
    return [e["puuid"] for e in entries if e.get("puuid")]

# Get MATCH IDS for a given PUUID
def get_match_ids(puuid, count=MATCHES_PER_PLAYER):
    url = f"https://{REGION}.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids"
    params = {"count": count, "queue": 1100}  # ranked TFT only
    r = riot_get(url, params)

    if r.status_code != 200:
        logging.warning(f"Failed match list for PUUID {puuid}")
        return []

    return r.json()

# Fetch a match JSON and save it
def fetch_and_save_match(match_id):
    out_path = SAVE_DIR / f"{match_id}.json"

    if out_path.exists():
        return

    url = f"https://{REGION}.api.riotgames.com/tft/match/v1/matches/{match_id}"
    r = riot_get(url)

    if r.status_code == 200:
        with open(out_path, "w") as f:
            json.dump(r.json(), f, indent=2)
        logging.info(f"Saved {match_id}")
    else:
        logging.warning(f"Match {match_id} failed with status {r.status_code}")


# MAIN SCRAPER
def main():
    logging.info("Starting Challenger + GM TFT scraper...")

    # 1) Fetch league PUUID pools
    challenger = get_challenger_puuids()
    grandmaster = get_grandmaster_puuids()

    logging.info(f"Challenger count: {len(challenger)}")
    logging.info(f"Grandmaster count: {len(grandmaster)}")

    #Rand samp 250 Challenger and 250 GM players
    challenger_sample = random.sample(challenger, min(250, len(challenger)))
    gm_sample = random.sample(grandmaster, min(250, len(grandmaster)))

    all_puuids = challenger_sample + gm_sample
    random.shuffle(all_puuids)

    #For each PUUID: fetch match list
    for i, puuid in enumerate(all_puuids, start=1):
        logging.info(f"Player {i}/{len(all_puuids)}: {puuid}")

        match_ids = get_match_ids(puuid)

        for m in match_ids:
            fetch_and_save_match(m)

    logging.info("Scrape complete.")

#
if __name__ == "__main__":
    main()

In [4]:
#once we have raw json, combine them into one pandas df. This will be the final raw dataframe for our data cleansing step

import pandas as pd
import os

json_directory = 'data/raw_matches' 
for filename in os.listdir(json_directory):
    if filename.endswith('.json'):
        filepath = os.path.join(json_directory, filename)
    try:
        # Read the JSON file into a DataFrame
        df = pd.read_json(filepath, lines=True)
    except ValueError as e:
        print(f"Error reading {filename}: {e}")
if dfs: # Check if there are any DataFrames to concatenate
    merged_df = pd.concat(dfs, ignore_index=True)
    print("Merge successful:")
    print(merged_df.head())
else:
    print("No JSON files found or processed.")

Error reading NA1_5412752266.json: Expected object or value
Error reading NA1_5414040301.json: Expected object or value
Error reading NA1_5408286501.json: Expected object or value
Error reading NA1_5409963945.json: Expected object or value
Error reading NA1_5405238773.json: Expected object or value
Error reading NA1_5410457390.json: Expected object or value
Error reading NA1_5412251146.json: Expected object or value
Error reading NA1_5412454758.json: Expected object or value
Error reading NA1_5406884277.json: Expected object or value
Error reading NA1_5410824180.json: Expected object or value
Error reading NA1_5418461450.json: Expected object or value
Error reading NA1_5412996164.json: Expected object or value
Error reading NA1_5411109172.json: Expected object or value
Error reading NA1_5412013147.json: Expected object or value
Error reading NA1_5417633990.json: Expected object or value
Error reading NA1_5414774120.json: Expected object or value
Error reading NA1_5406286492.json: Expec

NameError: name 'dfs' is not defined